In [2]:
# import libraries
import boto3
import os
from datetime import datetime, timedelta
from botocore import UNSIGNED
from botocore.client import Config
import xarray as xr
import gzip
import shutil
import logging
import pygrib
import numpy as np
import zarr
import io
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Suppress ECCODES warnings
logging.getLogger('eccodes').setLevel(logging.ERROR)
os.environ["ECCODES_WARNINGS"] = "0"
os.environ["ECCODES_LOG"] = "/dev/null"
import tempfile


In [3]:
single_date_file_output_prefix = '/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/'

### 1. download the original s3 file and merge it into 1 single file for SHSR/Density/Prob


In [4]:
def list_files_in_folder(bucket_name, folder_name):
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED), region_name='us-east-1')
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=folder_name)
    
    # Get all the files in the folder
    file_keys = []
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                file_keys.append(obj['Key'])
    return file_keys

def list_prob_files_in_folder(bucket_name, folder_name):
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED), region_name='us-east-1')
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=folder_name)
    
    # Get all the files in the folder
    file_keys = []
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                if obj['Key'][-13:-11] in("00"):
                    # print(obj['Key'])
                    file_keys.append(obj['Key'])
    return file_keys

def make_negative_val_as_0(ds):
    
    ds['tmp'] = ds.unknown.fillna(value=0) 
    ds_negative = ds.where(ds['tmp'] >=0,0)
    ds_positive = ds.where(ds['tmp']> 0)
    ds = xr.merge([ds_negative, ds_positive])
    data_dropped = ds.drop_vars("tmp")
    return data_dropped
    
def ct_prob_over_threshold_new(ds, ratio):


    one_percent = ratio * ratio / 100
    five_percent = ratio * ratio * 5 / 100
    fifty_percent = ratio * ratio * 50 / 100
    ds['prob_33_ct_gt_1'] = (ds['ds_coarsened_count_33'] > 1).astype(int)
    ds['prob_33_ct_gt_1_per'] = (ds['ds_coarsened_count_33'] > one_percent).astype(int)
    ds['prob_33_ct_gt_5_per'] =(ds['ds_coarsened_count_33'] > five_percent).astype(int)
    ds['prob_33_ct_gt_50_per'] = (ds['ds_coarsened_count_33'] > fifty_percent).astype(int)

    ds['prob_50_ct_gt_1'] = (ds['ds_coarsened_count_50'] > 1).astype(int)
    ds['prob_50_ct_gt_1_per'] = (ds['ds_coarsened_count_50'] > one_percent).astype(int)
    ds['prob_50_ct_gt_5_per'] = (ds['ds_coarsened_count_50'] > five_percent).astype(int)
    ds['prob_50_ct_gt_50_per'] = (ds['ds_coarsened_count_50'] > fifty_percent).astype(int)


def coarsen_001_025(ds_interp, expend_ratio):
    # in our case the expend ration should be 25
    coarsen_factor = {'latitude': expend_ratio, 'longitude': expend_ratio}

    
    # Sum
    ds_coarsened_sum = ds_interp.coarsen(coarsen_factor, boundary='trim').sum()
    new_var_names_sum = {var_name: f"sum_{var_name}" for var_name in ds_coarsened_sum.data_vars}
    ds_coarsened_sum = ds_coarsened_sum.rename(new_var_names_sum)
    
    # Max
    ds_coarsened_max = ds_interp.coarsen(coarsen_factor, boundary='trim').max()
    new_var_names_max = {var_name: f"max_{var_name}" for var_name in ds_coarsened_max.data_vars}
    ds_coarsened_max = ds_coarsened_max.rename(new_var_names_max)
    
    # Avg
    ds_coarsened_avg = ds_interp.coarsen(coarsen_factor, boundary='trim').mean()
    new_var_names_avg = {var_name: f"avg_{var_name}" for var_name in ds_coarsened_avg.data_vars}
    ds_coarsened_avg = ds_coarsened_avg.rename(new_var_names_avg)
    

    # Merge sum and max datasets
    ds_coarsened = xr.merge([ds_coarsened_sum, ds_coarsened_max, ds_coarsened_avg])
    ds_coarsened_filled = ds_coarsened.fillna(0)

    return ds_coarsened_filled

def read_and_convert_s3_file(bucket_name, folder_name, output_zarr_path, append_flag, var_name):
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED), region_name='us-east-1')
    if var_name =="probility":
        file_keys= list_prob_files_in_folder(bucket_name, folder_name)
    else:
        file_keys = list_files_in_folder(bucket_name, folder_name)
    
    if not file_keys:
        print(f"No data available for {folder_name.split('/')[-2]}.")
        return
    
    for file_key in file_keys:
        try:
            compressed_content = s3_client.get_object(Bucket=bucket_name, Key=file_key)['Body'].read()
            
            with gzip.GzipFile(fileobj=io.BytesIO(compressed_content)) as gz:
                grib_data = gz.read()
            
            with tempfile.NamedTemporaryFile(delete=False, suffix='.grib2') as temp_file:
                temp_file.write(grib_data)
                temp_file_path = temp_file.name
            
            ds = xr.open_dataset(temp_file_path, engine='cfgrib')
            ds = ds.expand_dims('time')
            ds = make_negative_val_as_0(ds)

            
            ds = ds.rename({"unknown": var_name})
            # if "prob" in var_name:
            #     ct_prob_over_threshold_new(ds, 10)
            # else: 
            #     ds = coarsen_001_025(ds, 10)
 
            if (not os.path.exists(output_zarr_path)) or (append_flag is False):
                ds.to_zarr(output_zarr_path, mode='w')
                append_flag = True
            else:
                ds.to_zarr(output_zarr_path, mode='a', append_dim='time')
            
            os.remove(temp_file_path)
        
        except Exception as e:
            print(f"Error processing file {file_key}: {e}")
            continue

def daterange(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y%m%d')
    end_date = datetime.strptime(end_date, '%Y%m%d')
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

def download_s3_file_per_day(bucket_name, data_type, var_name, start_date_str, end_date_str, save_var_name):

    for single_date in daterange(start_date_str, end_date_str):
        date_str = single_date.strftime('%Y%m%d')
        output_zarr_path = f"/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/{save_var_name}/{save_var_name}_{date_str}.zarr"
        folder_name = f'CONUS/{data_type}/{date_str}/'
        read_and_convert_s3_file(bucket_name, folder_name, output_zarr_path, False, var_name)
        print(start_date_str)


In [5]:
def first_download_single_day_file():
    bucket_name = 'noaa-mrms-pds'
    data_type = 'SeamlessHSR_00.00'
    var_name = "shsr"
    start_date_str = '20230801'
    end_date_str = '20230807'
    save_var_name = 'shsr002'
    
    download_s3_file_per_day(bucket_name, data_type, var_name, start_date_str, end_date_str,save_var_name)

  

In [6]:
# first_download_single_day_file()

### 2. Merge Single day file into 1 merged Zarr file

In [7]:
def toHour(ds):
    ds = ds.sortby('time')
    ds = ds.sel(time=~ds.get_index('time').duplicated())
    variables_to_resample = ds.data_vars
    # resample shsr
    resampled = {}
    for var in variables_to_resample:
        resampled[f'{var}_1hr_max'] = ds[var].resample(time='1H').max()
        resampled[f'{var}_1hr_min'] = ds[var].resample(time='1H').min()
        resampled[f'{var}_1hr_std'] = ds[var].resample(time='1H').std()
        resampled[f'{var}_1hr_sum'] = ds[var].resample(time='1H').sum()
    # merge it back to 1 zarr
    ds_resampled = xr.Dataset(resampled)
    return ds_resampled

def daterange(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, '%Y%m%d')
    end_date = datetime.strptime(end_date_str, '%Y%m%d')
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)
        

def merge_to_1_zarr(var_name, start_date_str,end_date_str, append_flag,output_file_path):
    
    for single_date in daterange(start_date_str, end_date_str):
        date_str = single_date.strftime('%Y%m%d')
        file_path = f"/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/{var_name}/{var_name}_{date_str}.zarr"
        
        try:
            ds = xr.open_zarr(file_path)
            ds = ds.drop_vars('valid_time')
            ds = toHour(ds)
            
            if (not os.path.exists(output_file_path)) or (append_flag is False):
                ds.to_zarr(output_file_path, mode='w')
                append_flag = True
            else:
                ds.to_zarr(output_file_path, mode='a', append_dim='time')
            print(date_str, "completed")
        except Exception as e:
            print(f"Error processing {date_str}: {e}")
            

def select_time(ds, start_date, end_date):
    time_range = slice(start_date, end_date)
    ds = ds.sel(time = time_range)
    return ds

def round_up_lat_lon(ds, ratio):
    new_longitude = np.around(ds["longitude"] / ratio) * ratio
    new_latitude = np.around(ds["latitude"] / ratio) * ratio

    ds_interp = ds.interp(
        latitude=new_latitude, longitude=new_longitude, method="nearest"
    )

    ds_interp["longitude"] = np.around(ds_interp["longitude"] // ratio) * ratio
    ds_interp["latitude"] = np.around(ds_interp["latitude"] // ratio) * ratio
    return ds_interp

def merge_ds_to_df_all(ds_list, start_date,end_date,ratio):
    
    for i  in range(len(ds_list)):
    
        # ds_list[i] = select_time(ds_list[i],start_date,end_date)
        # ds_list[i] = round_up_lat_lon(ds_list[i], ratio)
        # print(ds_list[i])
        # if 'sum_shsr_1hr_sum' in ds_list[i].data_vars:
        #     ds_list[i] = ds_list[i].where(ds_list[i]['sum_shsr_1hr_sum'] > 0, drop=True)
        ds_list[i] = ds_list[i].to_dataframe()
        ds_list[i] = ds_list[i].set_index(['time','latitude','longitude'])
    
    _path = "/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/"
    if not os.path.exists(_path):
        os.makedirs(_path)
   
    df_merged = ds_list[0].merge(ds_list[1], on=['time','latitude','longitude']).merge(ds_list[2], on=['time','latitude','longitude'])
    # df_filtered = df_merged[df_merged['sum_shsr_1hr_sum'] > 0]
    # df_filtered = df_merged
    df_merged.to_parquet(os.path.join(_path, '2022Train_w_probHr_002.parquet'))
    return df_merged






In [8]:
def second_merge_into_one_zarr_for_3_categories():
    var_name = "shsr002"
    start_date_str = '20220801'
    end_date_str = '20220807'
    # output_file_path = f'/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/20220801_20220807_{var_name}.zarr'
    # merge_to_1_zarr(var_name,start_date_str,end_date_str,False,output_file_path)
    
    t_ds_density = xr.open_dataset("/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/20220801_20220807_density002.zarr")
    t_ds_shsr   = xr.open_dataset("/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/20220801_20220807_shsr002.zarr")
    t_ds_prob = xr.open_dataset("/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/20220801_20220807_prob002.zarr")
  
 
    ds_list = [t_ds_density,t_ds_shsr,t_ds_prob]
    start_date ='2023-08-01'
    end_date = '2023-08-07'
    ratio = 1
    training_merged_df = merge_ds_to_df_all(ds_list, start_date,end_date,ratio)
    


In [9]:
# x = pd.read_parquet("/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/2022Train_w_probHr_002.parquet")
# print(x)

In [10]:
second_merge_into_one_zarr_for_3_categories()

/opt/homebrew/lib/python3.12/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/opt/homebrew/lib/python3.12/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/opt/homebrew/lib/python3.12/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


: 

### 3. Add Static Var + SHSR In Pre/Post Hour

In [85]:
def calculate_geographic_local_time(longitude, utc_datetime):
    timezone_offset = timedelta(hours=longitude / 15)
    local_time = utc_datetime + timezone_offset
    return local_time

def floor_to_nearest_hour(dt):
    return dt.replace(minute=0, second=0, microsecond=0)

def get_season(latitude, local_time):
    month = local_time.month
    # Northern Hemisphere 
    if latitude >= 0:  
        if 3 <= month <= 5:
            return 'Spring'
        elif 6 <= month <= 8:
            return 'Summer'
        elif 9 <= month <= 11:
            return 'Autumn'
        else:
            return 'Winter'
    else:  # Southern Hemisphere
        if 3 <= month <= 5:
            return 'Autumn'
        elif 6 <= month <= 8:
            return 'Winter'
        elif 9 <= month <= 11:
            return 'Spring'
        else:
            return 'Summer'
        
def add_static_var(df):

    local_times = []
    local_seasons = []
    for index, _ in df.iterrows():
        time, latitude, longitude = index
        if longitude > 180: longitude -= 360
        if latitude >90: latitude-=180
        local_time = calculate_geographic_local_time(longitude, time)
        local_season = get_season(latitude,time)
        local_times.append(local_time)
        local_seasons.append(local_season)

    # 1. get local time
    df['local_time'] = local_times 
    df['local_time'] = df['local_time'].apply(floor_to_nearest_hour)
    # 2. add day of year
    df['day_of_year'] = df['local_time'].dt.dayofyear
    # 3. add time of day
    df['hour_of_day'] = df['local_time'].apply(lambda x: x.hour)
    # 4. add local season
    df['season'] = local_seasons
    df['isSummer'] = df['season'] == 'Summer'
    df['isSpring'] = df['season'] == 'Spring'
    df['isWinter'] = df['season'] == 'Winter'
    df['isAutumn'] = df['season'] == 'Autumn'
    df.sort_values(by='local_time', inplace=True)

    return df

def get_shsr1_lightning1(df):
    df_lightning_positive = df[df['sum_density_1hr_sum'] > 0]
    df_return = df_lightning_positive[df_lightning_positive["avg_shsr_1hr_sum"]>0]
    return df_return


def get_shsr1_lightning0(df):
    df_shsr_positive = df[df['avg_shsr_1hr_sum'] > 0]
    df_shsr_positive['sum_density_1hr_sum'] = df_shsr_positive['sum_density_1hr_sum'].fillna(value=0) 
    df_return = df_shsr_positive.dropna(subset=['sum_density_1hr_sum'])
    return df_return    


def get_shsr1_lightning1_test(df):
    df_lightning_positive = df[df['sum_density_1hr_sum'] > 0]
    df_return = df_lightning_positive[df_lightning_positive["avg_shsr_1hr_sum"]>0]
    return df_return


def get_shsr1_lightning0_test(df):
    df_shsr_positive = df[df['avg_shsr_1hr_sum'] > 0]
    df_shsr_positive['sum_density_1hr_sum'] = df_shsr_positive['sum_density_1hr_sum'].fillna(value=0) 
    df_return = df_shsr_positive.dropna(subset=['sum_density_1hr_sum'])
    return df_return    


def add_shsr_ct_vars(df):
    df['avg_shsr_1hr_sum_gt_30'] = df['avg_shsr_1hr_sum'] > 30
    df['avg_shsr_1hr_sum_gt_40'] = df['avg_shsr_1hr_sum'] > 40
    df['avg_shsr_1hr_sum_gt_50'] = df['avg_shsr_1hr_sum'] > 50
    
    return df

shsr_var = ["sum_shsr_1hr_sum","avg_shsr_1hr_max","avg_shsr_1hr_sum",
            "max_shsr_1hr_max","max_shsr_1hr_std","max_shsr_1hr_sum","sum_shsr_1hr_max",
            "sum_shsr_1hr_min","sum_shsr_1hr_std",
            ]

def add_SHSR_feature(df):
    vars = shsr_var
    for var in vars:
        # prev 4 hr
        df[f'{var}_prev_4hr_sum'] = df[var].shift(1).rolling(window=4, min_periods=1).sum()
        # after 2hr 
        df[f'{var}_after_2hr_sum'] = df[var].shift(-1)+df[var].shift(-2)
        # after 4hr 
        df[f'{var}_after_4hr_sum'] = df[var].shift(-1)+df[var].shift(-2)+df[var].shift(-3)+df[var].shift(-4)

    # weather_shsr_var = ['tcc','d2m','t2m','ws'] + total_variables
    tmp_Vars = ["sum_shsr_1hr_sum","avg_shsr_1hr_max","avg_shsr_1hr_sum",
                "max_shsr_1hr_max","max_shsr_1hr_std","max_shsr_1hr_sum","sum_shsr_1hr_max",
                "sum_shsr_1hr_min","sum_shsr_1hr_std","avg_shsr_1hr_sum_gt_30","avg_shsr_1hr_sum_gt_40",
                "avg_shsr_1hr_sum_gt_50","sum_density_1hr_sum"]
    for var in tmp_Vars:
        # 创建四小时前到一小时前的列
        for i in range(1, 5):
            df[f'{var}_at-{i}hr'] = df[var].shift(i)

        # 创建一小时后到三小时后的列
        if var == "sum_lightning_density_1hr_sum":
            continue
        for i in range(1, 4):
            df[f'{var}_at+{i}hr'] = df[var].shift(-i)

    df1 = df.fillna(0)
    return df1

def change_dtype_for_parquet(df_train):
    for column in df_train.columns:
        if df_train[column].dtype == 'object':
            if column == 'local_time':
                try:
                    # 将 'local_time' 列转换为 datetime64 类型
                    df_train[column] = pd.to_datetime(df_train[column], errors='coerce')
                    print(f"Converted column '{column}' to datetime.")
                except ValueError:
                    print(f"Column '{column}' cannot be converted to datetime and will be skipped.")
            elif column == 'season':
                try:
                    # 将 'season' 列转换为 category 类型
                    df_train[column] = df_train[column].astype('string')
                except ValueError:
                    print(f"Column '{column}' cannot be converted to category and will be skipped.")
            else:
                try:
                    # 尝试将其他列转换为 bool 类型
                    df_train[column] = df_train[column].astype('bool')
                except ValueError:
                    # 如果转换失败，则保留原类型并打印警告
                    print(f"Column '{column}' cannot be converted to bool and will be skipped.")
    return df_train


def third_generate_the_training_file_with_static_and_shsr_features():

    ds_train = pd.read_parquet("/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/2023Test_w_probHr_per1.parquet")
    df_train = ds_train.reset_index()
    df1 = get_shsr1_lightning1(df_train)
    df2 =  get_shsr1_lightning0(df_train)
    df1 = df1.reset_index(drop=True)
    df2 = df2.reset_index(drop=True)
    df_train_merged = pd.concat([df1, df2], axis=0,ignore_index=True)
    df_train_merged = df_train_merged.set_index(["time", "latitude", "longitude"])
    
    df_static = add_static_var(df_train_merged)
    df_static = df_static.dropna(subset=['avg_shsr_1hr_sum'])
    df_test_merged= add_shsr_ct_vars(df_static)
    df_static = add_SHSR_feature(df_static)
    print(df_test_merged.columns)    
    
    
    
    parquet_output_path = "/Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/test_merged_202409_prevpost_per1.parquet"
    df_test_merged.to_parquet(parquet_output_path, index=True)

    print(f"Data saved as Parquet at {parquet_output_path}")
    
    
    return df_test_merged



In [86]:
third_generate_the_training_file_with_static_and_shsr_features()

/var/folders/zf/my42n5_x09g47p6r08bz8_1w0000gn/T/ipykernel_21822/306381155.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{var}_at+{i}hr'] = df[var].shift(-i)
/var/folders/zf/my42n5_x09g47p6r08bz8_1w0000gn/T/ipykernel_21822/306381155.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{var}_at-{i}hr'] = df[var].shift(i)
/var/folders/zf/my42n5_x09g47p6r08bz8_1w0000gn/T/ipykernel_21822/306381155.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

Index(['avg_density_1hr_max', 'avg_density_1hr_min', 'avg_density_1hr_std',
       'avg_density_1hr_sum', 'heightAboveSea_x', 'max_density_1hr_max',
       'max_density_1hr_min', 'max_density_1hr_std', 'max_density_1hr_sum',
       'step_x',
       ...
       'avg_shsr_1hr_sum_gt_50_at+1hr', 'avg_shsr_1hr_sum_gt_50_at+2hr',
       'avg_shsr_1hr_sum_gt_50_at+3hr', 'sum_density_1hr_sum_at-1hr',
       'sum_density_1hr_sum_at-2hr', 'sum_density_1hr_sum_at-3hr',
       'sum_density_1hr_sum_at-4hr', 'sum_density_1hr_sum_at+1hr',
       'sum_density_1hr_sum_at+2hr', 'sum_density_1hr_sum_at+3hr'],
      dtype='object', length=170)
Data saved as Parquet at /Users/jesse/Desktop/OpenSource/Lightning-Prediction-ML/zarr_0_1/merged/test_merged_202409_prevpost_per1.parquet


avg_density_1hr_max  \
time                latitude longitude                        
2023-07-09 00:00:00 50.7     236.3                  0.01276   
                    43.7     237.9                  0.00000   
                             237.7                  0.00000   
                             237.6                  0.00000   
                    43.8     239.9                  0.00000   
...                                                     ...   
2023-07-15 16:00:00 41.2     288.4                  0.00000   
                             288.5                  0.00000   
                             288.6                  0.00000   
                    46.2     291.5                  0.00000   
                    45.6     294.5                  0.00000   

                                        avg_density_1hr_min  \
time                latitude longitude                        
2023-07-09 00:00:00 50.7     236.3                      0.0   
                    43.7     237.9                      0.0   
                             237.7                      0.0   
                             237.6                      0.0   
                    43.8     239.9                      0.0   
...                                                     ...   
2023-07-15 16:00:00 41.2     288.4                      0.0   
                             288.5                      0.0   
                             288.6                      0.0   
                    46.2     291.5                      0.0   
                    45.6     294.5                      0.0   

                                        avg_density_1hr_std  \
time                latitude longitude                        
2023-07-09 00:00:00 50.7     236.3                 0.006366   
                    43.7     237.9                 0.000000   
                             237.7                 0.000000   
                             237.6                 0.000000   
                    43.8     239.9                 0.000000   
...                                                     ...   
2023-07-15 16:00:00 41.2     288.4                 0.000000   
                             288.5                 0.000000   
                             288.6                 0.000000   
                    46.2     291.5                 0.000000   
                    45.6     294.5                 0.000000   

                                        avg_density_1hr_sum  heightAboveSea_x  \
time                latitude longitude                                          
2023-07-09 00:00:00 50.7     236.3                  0.35728               0.0   
                    43.7     237.9                  0.00000               0.0   
                             237.7                  0.00000               0.0   
                             237.6                  0.00000               0.0   
                    43.8     239.9                  0.00000               0.0   
...                                                     ...               ...   
2023-07-15 16:00:00 41.2     288.4                  0.00000               0.0   
                             288.5                  0.00000               0.0   
                             288.6                  0.00000               0.0   
                    46.2     291.5                  0.00000               0.0   
                    45.6     294.5                  0.00000               0.0   

                                        max_density_1hr_max  \
time                latitude longitude                        
2023-07-09 00:00:00 50.7     236.3                    1.276   
                    43.7     237.9                    0.000   
                             237.7                    0.000   
                             237.6                    0.000   
                    43.8     239.9                    0.000   
...                                                     ...   
2023-07-15 16:00:00 41